# Use pretrained model for test predictions

In [4]:
import torch

from utils import SR
from datasets import Audio, load_dataset
from transformers import AutoModelForCTC, Wav2Vec2Processor

dataset_name = "lucas-meyer/asr_af"
repo_name = "lucas-meyer/wav2vec2-xls-r-300m-asr_af"

dataset = load_dataset("audiofolder", data_dir="asr_af")
test_set = dataset["test"].cast_column("audio", Audio(sampling_rate=SR)).rename_column("transcription", "sentence")

Resolving data files:   0%|          | 0/3325 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/511 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/585 [00:00<?, ?it/s]

In [9]:
import os

import pandas as pd

from load_nchlt import load_nchlt

dataset_name = "nchlt_set"
os.makedirs(dataset_name, exist_ok=True)

csv_entries = []
csv_entries += load_nchlt(only_af=True, write_audio=True)
metadata = pd.DataFrame(csv_entries, columns=['file_name', 'transcription'])
metadata.to_csv(path_or_buf=os.path.join(dataset_name, "metadata.csv"), sep=",", index=False)
dataset = load_dataset("audiofolder", data_dir=dataset_name)
test_set = dataset["test"].cast_column("audio", Audio(sampling_rate=SR)).rename_column("transcription", "sentence")

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

KeyError: 'test'

In [ ]:
model = AutoModelForCTC.from_pretrained(repo_name)
processor = Wav2Vec2Processor.from_pretrained(repo_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
for i in range(100):
    input_values = processor(test_set[i]["audio"]["array"], sampling_rate=SR).input_values[0]
    input_dict = processor(input_values,
                           sampling_rate=SR,
                           return_tensors="pt",
                           padding=True)

    logits = model(input_dict.input_values).logits
    logits = logits.detach()
    pred_ids = torch.argmax(logits, dim=-1)[0]

    pred = processor.decode(pred_ids)
    true = test_set[i]["sentence"].lower()

    print(f"Test {i}:")
    print(f"  - pred: {pred}")
    print(f"  - true: {true}\n")

Test 0:
  - pred: waaram is dit positief negadief of neutraal
  - true: waarom is dit positief negatief of neutraal

Test 1:
  - pred: dus is die woord measurer baie naby aan leisure
  - true: dus is die woord measure baie naby aan leisure

Test 2:
  - pred: die muskiek groep zef maar zen het meer as 25 miljoen albums wêreldwyd verkoop
  - true: die muskiekgroep zef maar zen het meer as 25 miljoen albums wêreldwyd verkoop

Test 3:
  - pred: thus hab the wraw worldiness prepeared him for this day
  - true: thus had the raw wilderness prepared him for this day

Test 4:
  - pred: uiteindelik was daar geen meersprake van dosns en emerate nie
  - true: uiteindelik was daar geen meer sprake van dominees en emirate nie

Test 5:
  - pred: vyf en vyftig twee een en twintig
  - true: vyf en vyftig twee een en twintig

Test 6:
  - pred: dit is minus 1 grade celsius in boksburg
  - true: dit is minus 1 grade celsius in boksburg

Test 7:
  - pred: postma het om tot almal se verbasing losgemaak van 